In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
import pingouin as pg
import plotly.graph_objects as go
from os.path import join as pjoin
import plotly.express as px

sys.path.append("../..")
import circletrack_behavior as ctb
import plotting_functions as pf

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [ ]:
## Excluded mice ms18, ms22, ms28 due to failing to learn
## Set mouse lists and directory
mouse_list = [f'mf0{num}' for num in np.arange(1, 5)]
male = ['mf03', 'mf04']
behav_path = '../../../MultiCon_Behavior/MF_Pilot/output/behav/'
fig_path = '../../../MultiCon_Behavior/MF_Pilot/intermediate_figures/'
if not os.path.exists(fig_path):
    os.mkdir(fig_path)

## Group colors to change plots
group_colors = {'Female': 'darkgrey', 'Male': 'turquoise' }

In [ ]:
## Check whether trial trimes are different between groups
trial_times_dict = {'mouse': [], 'forward_trials': [], 'reverse_trials': [], 'group': [], 'session': []}
for mouse in mouse_list:
    mouse_path = pjoin(behav_path, mouse)
    group = 'Male' if mouse in male else 'Female'
    for session in os.listdir(mouse_path):
        behav_data = pd.read_feather(pjoin(mouse_path, f'{session}'))
        time_diff_forward, time_diff_reverse = ctb.calculate_trial_length(behav_data, forward_reverse=True, recalc_trials=False)
        trial_times_dict['mouse'].append(mouse)
        trial_times_dict['forward_trials'].append(time_diff_forward)
        trial_times_dict['reverse_trials'].append(time_diff_reverse)
        trial_times_dict['group'].append(group)
        trial_times_dict['session'].append(np.unique(behav_data['session'])[0])
## Convert to dataframe
trial_times = pd.DataFrame(trial_times_dict)

fixed_trials = []
for trials in trial_times['forward_trials'][trial_times['group'] == 'fixed']:
    fixed_trials = fixed_trials + trials

criteria_trials = []
for trials in trial_times['forward_trials'][trial_times['group'] == 'criteria']:
    criteria_trials = criteria_trials + trials

cr5_trials = []
for trials in trial_times['forward_trials'][trial_times['group'] == 'cr_5']:
    cr5_trials = cr5_trials + trials

In [ ]:
## Bar chart of trial times per mouse per session
trial_dict_session = {'mouse': [], 'avg_time': [], 'std': [], 'sem': [], 'group': [], 'session': []}
for mouse in np.unique(trial_times['mouse']):
    mouse_trials = trial_times[trial_times['mouse'] == mouse].reset_index()
    for idx, trials in enumerate(mouse_trials['forward_trials']):
        trial_dict_session['mouse'].append(mouse)
        trial_dict_session['avg_time'].append(np.mean(trials))
        trial_dict_session['std'].append(np.std(trials))
        trial_dict_session['sem'].append(np.std(trials)/len(trials))
        trial_dict_session['group'].append(np.unique(mouse_trials['group'])[0])
        trial_dict_session['session'].append(mouse_trials.loc[idx, 'session'])
mouse_trial_session_df = pd.DataFrame(trial_dict_session)
avg_session = mouse_trial_session_df.groupby(['group', 'session'], as_index=False).agg({'avg_time': ['mean', 'sem']})
avg_session_df = avg_session['avg_time']
avg_session_df.insert(0, 'group', avg_session['group'])
avg_session_df.insert(1, 'session', avg_session['session'])

In [ ]:
fig = pf.plot_group_averages(mouse_trial_session_df, y_col_name='avg_time', x_col_name='session', mouse_grouping_variables=['mouse', 'session', 'group'],
                             avg_grouping_variables=['group', 'session'], group_color_dict=group_colors, x_axis_order=['A', 'B'],
                             height=500, width=500, template='simple_white', legend_title_text='', 
                             yaxis_title='Average Trial Time (s)', xaxis_title='', font=dict(size=16))
fig.show()
fig.write_image(pjoin(fig_path, 'average_trial_time.png'))

In [ ]:
mouse_trial_session_df.mixed_anova(dv='avg_time', within='session', subject='mouse', between='group')

In [ ]:
## Number of trials
num_trials_dict = {'mouse': [], 'num_trials': [], 'group': [], 'session': []}
for mouse in np.unique(trial_times['mouse']):
    sub_data = trial_times[trial_times['mouse'] == mouse]
    for row, _ in sub_data.iterrows():
        num_trials_dict['mouse'].append(mouse)
        num_trials_dict['num_trials'].append(len(sub_data.loc[row, 'forward_trials']))
        num_trials_dict['group'].append(sub_data.loc[row, 'group'])
        num_trials_dict['session'].append(sub_data.loc[row, 'session'])
num_trials_df = pd.DataFrame(num_trials_dict)

In [ ]:
fig = pf.plot_group_averages(num_trials_df, y_col_name='num_trials', x_col_name='session', mouse_grouping_variables=['mouse', 'session', 'group'],
                             avg_grouping_variables=['group', 'session'], group_color_dict=group_colors, x_axis_order=['A', 'B'],
                             height=500, width=500, template='simple_white', legend_title_text='', 
                             yaxis_title='Number of Trials', xaxis_title='', font=dict(size=16))
fig.show()
fig.write_image(pjoin(fig_path, 'average_number_of_trials.png'))

In [ ]:
num_trials_df.mixed_anova(dv='num_trials', within='session', subject='mouse', between='group')

In [ ]:
## Probe Accuracy
lick_dict_probe = {'percent_correct': [], 'mouse': [], 'day': [], 'group': [], 'session': [], 'probe_length': [], 'num_licks': [], 'session_pc': []}
for mouse in mouse_list:
    mouse_path = pjoin(behav_path, mouse)
    group = 'Male' if mouse in male else 'Female'
    for idx, session in enumerate(os.listdir(mouse_path)):
        behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
        if any(behav['probe']):
            behav_probe = behav[behav['probe']]
            behav_no_probe = behav[~behav['probe']]
            reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
            ## Percent correct licking
            pc = ctb.lick_accuracy(behav_probe, port_one=reward_one, port_two=reward_two, by_trials=False)
            session_pc = ctb.lick_accuracy(behav_no_probe, port_one=reward_one, port_two=reward_two, by_trials=False)
            lick_dict_probe['percent_correct'].append(pc)
            lick_dict_probe['mouse'].append(mouse)
            lick_dict_probe['day'].append(idx+1)
            lick_dict_probe['group'].append(group)
            lick_dict_probe['session'].append(np.unique(behav['session'])[0])
            lick_dict_probe['probe_length'].append(round(behav_probe['t'].to_numpy()[-1] - behav_probe['t'].to_numpy()[0]))
            lick_dict_probe['num_licks'].append(len(behav_probe[behav_probe['lick_port'] != -1]))
            lick_dict_probe['session_pc'].append(session_pc)
        else:
            next
## Convert to dataframe
probe_df = pd.DataFrame(lick_dict_probe)
# First and last day in each context
last = pd.DataFrame()
for mouse in mouse_list:
    mouse_data = probe_df[probe_df['mouse'] == mouse].reset_index(drop=True)
    index_list = ctb.pick_context_day(mouse_data, day_index=-1, contexts=['A', 'B'])
    sub_data = mouse_data.loc[index_list, :]
    last = pd.concat([last, sub_data])          
last['licks_sec'] = last['num_licks'] / last['probe_length']

In [ ]:
probe_df

In [ ]:
## Bar chart of probe accuracy per mouse on the last day within a context
subset = last[(last['mouse'] == 'mf02') | (last['mouse'] == 'mf03')]
fig = pf.plot_group_averages(subset, y_col_name='percent_correct', x_col_name='session', mouse_grouping_variables=['mouse', 'session', 'group'],
                             avg_grouping_variables=['group', 'session'], group_color_dict=group_colors, x_axis_order=['A', 'B'],
                             height=500, width=500, template='simple_white', legend_title_text='', 
                             yaxis_title='Probe Accuracy (%)', xaxis_title='', font=dict(size=16))
fig.add_hline(y=25, line_dash='dash', line_width=2, line_color='black')
fig.show()
fig.write_image(pjoin(fig_path, 'probe_accuracy_last_day.png'))

In [ ]:
## Number of licks during the probe for each context
fig = pf.plot_group_averages(subset, y_col_name='licks_sec', x_col_name='session', mouse_grouping_variables=['mouse', 'session', 'group'],
                             avg_grouping_variables=['group', 'session'], group_color_dict=group_colors, x_axis_order=['A', 'B'],
                             height=500, width=500, template='simple_white', legend_title_text='', 
                             yaxis_title='Licks/s During Probe', xaxis_title='', font=dict(size=16))
fig.show()
fig.write_image(pjoin(fig_path, 'licks_sec_probe.png'))

In [ ]:
## Probe Accuracy vs Probe Length for last day in each context
session_list = ['A', 'B']
fig = pf.custom_graph_template(x_title='Probe Length (s)', y_title='', rows=1, columns=2, 
                               width=1500, height=500, titles=['A', 'B'], shared_x=True, shared_y=True, font_size=17)
for idx, session in enumerate(session_list):
    sub_f = subset[(subset['session'] == session) & (subset['group'] == 'Female')]
    sub_c = subset[(subset['session'] == session) & (subset['group'] == 'Male')]

    fig.add_trace(go.Scatter(x=sub_f['probe_length'], y=sub_f['percent_correct'], mode='markers',
                             showlegend=False, name='Female', marker_line_width=1, legendgroup='Female',
                             marker_color=group_colors['Female']), row=1, col=idx+1)
    fig.add_trace(go.Scatter(x=sub_c['probe_length'], y=sub_c['percent_correct'], mode='markers',
                             showlegend=False, name='Male', marker_line_width=1, legendgroup='Male',
                             marker_color=group_colors['Male']), row=1, col=idx+1)
fig.update_yaxes(title='Probe Accuracy (%)', row=1, col=1)
fig['data'][0]['showlegend'] = True
fig['data'][1]['showlegend'] = True
fig.show()
fig.write_image(pjoin(fig_path, 'probe_accuracy_vs_length_last_day.png'))

In [ ]:
## Number of rewards and percent correct
result_dict = {'mouse': [], 'rewards': [], 'percent_correct': [], 'group': [], 'session': []}
for mouse in mouse_list:
    mouse_path = pjoin(behav_path, mouse)
    group = 'Male' if mouse in male else 'Female'
    for session in os.listdir(mouse_path):
        behav_data = pd.read_feather(pjoin(mouse_path, f'{session}'))
        behav_data = behav_data[~behav_data['probe']]
        reward_one, reward_two = np.unique(behav_data['reward_one'])[0], np.unique(behav_data['reward_two'])[0]
        pc = ctb.lick_accuracy(behav_data, port_one=reward_one, port_two=reward_two, by_trials=False)
        result_dict['mouse'].append(mouse)
        result_dict['rewards'].append(behav_data[behav_data['water']].shape[0])
        result_dict['percent_correct'].append(pc)
        result_dict['group'].append(group)
        result_dict['session'].append(np.unique(behav_data['session'])[0])
rewards = pd.DataFrame(result_dict)
last_rewards = pd.DataFrame()
for mouse in mouse_list:
    mouse_data = rewards[rewards['mouse'] == mouse].reset_index(drop=True)
    index_list = ctb.pick_context_day(mouse_data, day_index=-1, contexts=['A', 'B'])
    sub_data = mouse_data.loc[index_list, :]
    last_rewards = pd.concat([last_rewards, sub_data])

In [ ]:
## Plot rewards by session on last day in context
subset_rewards = last_rewards[(last_rewards['mouse'] == 'mf02') | (last_rewards['mouse'] == 'mf03')]
fig = pf.plot_group_averages(subset_rewards, y_col_name='rewards', x_col_name='session', mouse_grouping_variables=['mouse', 'session', 'group'],
                             avg_grouping_variables=['group', 'session'], group_color_dict=group_colors, x_axis_order=['A', 'B'],
                             height=500, width=500, template='simple_white', legend_title_text='', 
                             yaxis_title='Total Rewards', xaxis_title='', font=dict(size=16))
fig.show()
fig.write_image(pjoin(fig_path, 'total_rewards_last_day.png'))

In [ ]:
behav_data = pd.read_feather(pjoin(behav_path, 'mf03/mf03_6.feat'))
pf.plot_circle_position(behav_data, x_title='X Position', y_title='Y Position', downsample_factor=50)